In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from main import *

100%|██████████| 986/986 [00:00<00:00, 6966.87it/s]


In [3]:
print(config, sep='\n')

{'model_name': 'LSTM_3', 'feature': 'back-to-origins', 'max_len': 42, 'min_freq_src': 5, 'min_freq_trg': 5, 'embedding_dim': 192, 'hidden_size': 320, 'num_layers': 3, 'num_epochs': 15, 'weight_decay': 2e-05, 'label_smoothing': 0.1, 'dropout_emb': 0.2, 'dropout_enc': 0.4, 'dropout_dec': 0.4, 'dropout_attention': 0.15, 'learning_rate': 0.0008, 'gamma': 0.2, 'patience': 1, 'threshold': 0.0005, 'batch_size': 128, 'use_tf': False, 'tf_from_epoch': 0, 'tf_start': 0.9, 'tf_decrease': 0.02, 'src_vocab_size': 24991, 'trg_vocab_size': 18710}


In [4]:
device='cuda'

In [5]:
vocab_src, vocab_trg, train_dataset, val_dataset

(<dataset.Vocab at 0x2722527acf0>,
 <dataset.TranslationDataset at 0x2722522b110>)

In [ ]:
from dataset2 import TrainDataLoader, TestDataLoader
from config import filenames, folders
from lstm3 import LSTM_3

In [7]:
unk_idx, pad_idx, bos_idx, eos_idx = 0, 1, 2, 3
device = 'cuda'

train_loader = TrainDataLoader(train_dataset, shuffle=True)
val_loader = TestDataLoader(val_dataset, shuffle=False)

weights_filename = folders['weights'] + 'LSTM_3-back-to-origins-30.0m-15epoch.pt'

model = LSTM_3(config=config, weights_filename=weights_filename).to(device)
print(model)
print(model.count_parameters())

LSTM_3(
  (src_embedding): Embedding(24991, 192, padding_idx=1)
  (trg_embedding): Embedding(18710, 192, padding_idx=1)
  (emb_dropout): Dropout(p=0.2, inplace=True)
  (enc_dropout): Dropout(p=0.4, inplace=True)
  (dec_dropout): Dropout(p=0.4, inplace=True)
  (attention_dropout): Dropout(p=0.15, inplace=True)
  (encoder): LSTM(192, 320, num_layers=3, batch_first=True, dropout=0.4, bidirectional=True)
  (encoder_output_proj): Linear(in_features=640, out_features=320, bias=True)
  (decoder): LSTM(192, 320, num_layers=3, batch_first=True, dropout=0.4)
  (encoder_hidden_proj): ModuleList(
    (0-2): 3 x Linear(in_features=640, out_features=320, bias=True)
  )
  (encoder_cell_proj): ModuleList(
    (0-2): 3 x Linear(in_features=640, out_features=320, bias=True)
  )
  (fc): Linear(in_features=640, out_features=18710, bias=True)
)
30362262


In [8]:
from submission import get_bleu

In [55]:
from dataset import RawDataset
raw_dataset = RawDataset(filenames['test_trg'])
get_bleu(model, val_loader, vocab_trg, raw_dataset, True, 1, border=0.01/100)

 75%|███████▌  | 3/4 [00:09<00:03,  3.16s/it]

correct:	 9704 5192 3113 1896
total:		 14483 13497 12511 11533
sinergy:	32.04
BP:		0.673
67.0/38.5/24.9/16.4


np.float64(21.56)

In [56]:
print(model)

LSTM_3(
  (src_embedding): Embedding(24991, 192, padding_idx=1)
  (trg_embedding): Embedding(18710, 192, padding_idx=1)
  (emb_dropout): Dropout(p=0.2, inplace=True)
  (enc_dropout): Dropout(p=0.4, inplace=True)
  (dec_dropout): Dropout(p=0.4, inplace=True)
  (attention_dropout): Dropout(p=0.15, inplace=True)
  (encoder): LSTM(192, 320, num_layers=3, batch_first=True, dropout=0.4, bidirectional=True)
  (encoder_output_proj): Linear(in_features=640, out_features=320, bias=True)
  (decoder): LSTM(192, 320, num_layers=3, batch_first=True, dropout=0.4)
  (encoder_hidden_proj): ModuleList(
    (0-2): 3 x Linear(in_features=640, out_features=320, bias=True)
  )
  (encoder_cell_proj): ModuleList(
    (0-2): 3 x Linear(in_features=640, out_features=320, bias=True)
  )
  (fc): Linear(in_features=640, out_features=18710, bias=True)
)


In [48]:
from submission import make_submission
from dataset import SubmissionDataLoader, SubmissionDataset
raw_dataset = RawDataset(filenames['submission_src'])
submission_dataset = SubmissionDataset(filenames['submission_src'], vocab=vocab_src)
submission_loader = SubmissionDataLoader(submission_dataset)
make_submission(model, submission_loader=submission_loader, raw_dataset=raw_dataset, vocab_trg=vocab_trg, filenames=filenames)

100%|██████████| 24/24 [00:24<00:00,  1.02s/it]


src:             sie wollen die menschen in angst leben lassen .
trg:             they want to make people live in fear .
pred:            they want to live people in fear .
pred-beam:       they want people to live in fear .
sinergy:        37.15
BP:             0.882
100.0/57.1/33.3/10.0
bleu: 32.78
sinergy:        44.18
BP:             0.882
100.0/57.1/33.3/20.0
bleu beam: 38.99

In [50]:
raw_dataset = RawDataset(filenames['test_trg'])
float(get_bleu(model, dataloader=val_loader, vocab_trg=vocab_trg, device=device, raw_dataset=raw_dataset, use_beam=True, beam_width=1))

 75%|███████▌  | 3/4 [00:09<00:03,  3.20s/it]

correct:	 11551 6301 3720 2242
total:		 17863 16877 15891 14913
sinergy:	30.36
BP:		0.876
64.7/37.3/23.4/15.0


26.6

### greedy
sinergy:	29.56

BP:		    0.955

61.3/36.2/23.0/14.9

correct:	 11913 6661 4013 2459

total:	 19461 18475 17490 16510

bleu: 28.34

### beam
sinergy:	24.19

BP:		0.977

54.7/30.3/18.3/11.3

correct:	 10826 5699 3255 1898

total:	 19774 18788 17803 16826


bleu:   23.65

In [12]:
from submission import bleu

In [13]:
src, trg = val_dataset[20]

In [14]:
" ".join(vocab_trg.decode(model.inference(src.unsqueeze(0)).squeeze(0)))
" ".join(vocab_trg.decode(trg))

'we were scared , but still , school was where we wanted to be .'

In [15]:
bleu(vocab_trg.decode(model.inference(src.unsqueeze(0)).squeeze(0)), vocab_trg.decode(trg))

np.float64(11.726550850719802)

In [49]:
model.demonstrate(val_loader, vocab_src, vocab_trg, device='cuda', examples=3, wait=0, verbose=0)

src:		 als ich <NUM> jahre alt war , wurde ich eines morgens von den klängen heller freude geweckt .
trg:		 when i was <NUM> , i remember waking up one morning to the sound of joy in my house .
pred:		 when i was <NUM> years old , i was going to be able to get the message to the outside of the message .
pred-beam:	 when i was <NUM> years old , one morning , i was one morning .
sinergy:	14.07
BP:		1
41.7/21.7/9.1/4.8
bleu:		14.07
sinergy:	22.89
BP:		0.717
60.0/35.7/15.4/8.3
bleu beam:	16.40

src:		 mein vater hörte sich auf seinem kleinen , grauen radio die der bbc an .
trg:		 my father was listening to bbc news on his small , gray radio .
pred:		 my father was listening to his little , terrifying radio , the baby 's favorite of the bbc .
pred-beam:	 my dad stopped
sinergy:	22.54
BP:		1
52.6/22.2/17.6/12.5
bleu:		22.54
sinergy:	27.52
BP:		0.026
33.3/25.0/25.0/0.0
bleu beam:	0.70

src:		 er sah sehr glücklich aus , was damals ziemlich ungewöhnlich war , da ihn die nachrichten meistens .


In [26]:
import torch
torch.cuda.ipc_collect()

In [21]:
from submission import bleu, get_precisions
ref = "there was a big smile on his face which was unusual then , because the news mostly depressed him ."
pred = "he looked very happy , which was quite unusual , because it 's the news mostly ."
bleu(ref.split(), pred.split())

np.float64(11.168140627821579)

In [27]:
import torch
pred_idx = model.inference(torch.tensor(vocab_src.encode(ref.split()), dtype=torch.long, device='cuda').unsqueeze(0))

In [29]:
vocab_trg.decode(pred_idx[0])

['and', 'what', 'was', 'a', 'big', 'favorite']

In [19]:
get_precisions(ref.split(), pred.split())

(array([9., 4., 1., 0.]), array([17., 16., 15., 14.]))

In [ ]:
from submission import bleu
def export(filename, txt):
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(txt)

ref_filename = '../submission/a.txt'
pred_filename = '../submission/b.txt'
def get_bleu_txt(pred, ref):
    export(pred_filename, pred)
    export(ref_filename, ref)
    print(pred)
    print(ref)

    print('custom: ', bleu(ref=ref.split(), c=pred.split(), verbose=True))
    # print('correct: ', eval_bleu(pred_filename, ref_filename))

get_bleu_txt(pred, ref)

he looked very happy , which was quite unusual , because it 's the news mostly .
there was a big smile on his face which was unusual then , because the news mostly depressed him .
sinergy:	13.32
BP:		0.838
52.9/25.0/6.7/3.6
custom:  11.168140627821579


In [10]:
model.demonstrate(val_loader, vocab_src, vocab_trg, examples=5, wait=3)

NameError: name 'model' is not defined

In [ ]:
len(pred.split())

17

In [ ]:
raw_submission = RawDataset(filenames['submission_src'])

In [ ]:
from submission import make_submission
from dataset import SubmissionDataset, SubmissionDataLoader
submission_dataset = SubmissionDataset(filenames['submission_src'], vocab_src, device=device)
ldr = SubmissionDataLoader(submission_dataset)
# make_submission(model, ldr, vocab_trg, filenames, device=device, raw_dataset=raw_submission)

In [ ]:
src, trg = val_dataset[0]
print(vocab_src.decode(src))

['als', 'ich', '<NUM>', 'jahre', 'alt', 'war', ',', 'wurde', 'ich', 'eines', 'morgens', 'von', 'den', 'klängen', 'heller', 'freude', 'geweckt', '.']


In [ ]:
val_loader = TestDataLoader(val_dataset, shuffle=False, batch_size=3)

In [ ]:
for src, trg in val_loader:
    model.inference_beam(src, verbose=True, max_len=5, vocab_trg=vocab_trg)
    break

k: 5	B: 3

step:0
beam tokens:
index in batch: 0
<BOS>
<BOS>
<BOS>
<BOS>
<BOS>

index in batch: 1
<BOS>
<BOS>
<BOS>
<BOS>
<BOS>

index in batch: 2
<BOS>
<BOS>
<BOS>
<BOS>
<BOS>

beam probs:
[[100.   0.   0.   0.   0.]
 [100.   0.   0.   0.   0.]
 [100.   0.   0.   0.   0.]]

step:1
beam tokens:
index in batch: 0
<BOS> when
<BOS> and
<BOS> by
<BOS> at
<BOS> i

index in batch: 1
<BOS> my
<BOS> and
<BOS> dad
<BOS> the
<BOS> i

index in batch: 2
<BOS> he
<BOS> and
<BOS> it
<BOS> so
<BOS> now

beam probs:
[[81.9  1.8  1.4  1.2  1.2]
 [81.9  5.   1.5  1.2  1.1]
 [54.9 16.5  4.5  2.2  1.4]]

step:2
beam tokens:
index in batch: 0
<BOS> when i
<BOS> and when
<BOS> by <NUM>
<BOS> i was
<BOS> at <NUM>

index in batch: 1
<BOS> my father
<BOS> my dad
<BOS> and my
<BOS> my old
<BOS> the father

index in batch: 2
<BOS> he looked
<BOS> he saw
<BOS> and he
<BOS> he was
<BOS> it looked

beam probs:
[[74.1  1.5  0.8  0.6  0.4]
 [49.7 20.2  4.1  0.6  0.6]
 [27.3 15.  12.2  4.5  2.2]]

step:3
beam tokens:


In [ ]:
from dataset import TestDataLoader
ldr = TestDataLoader(val_dataset, batch_size=2)
src, trg = ldr[0]
src.size(1)

20

In [ ]:
import torch
from torch.nn.functional import softmax
trg_words = ['<BOS>', 'i', 'was', '<NUM>', 'years', 'i', 'was']
i = len(trg_words) - 1
trg_pred = torch.tensor([vocab_trg.encode_word(word) for word in trg_words], dtype=torch.long).unsqueeze(0)
predictions = model.inference(src, device='cuda')
print(vocab_trg.decode(trg_pred.squeeze(0).numpy()))
for idx in predictions[0]:
    print(vocab_trg.decode_idx(idx.item()), end=' ')

['i', 'was', '<NUM>', 'years', 'i', 'was']
<BOS> when i was <NUM> years old , i was told one morning by the bright hollywood joy , and i was behind <EOS> 